In [1]:
import os 
import csv
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# For preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper 

import torch # For building the networks 
import torchtuples as tt # Some useful functions

from pycox.datasets import gbsg
from BandDeepHitSingle import BandedDeepHitSingle
from pycox.evaluation import EvalSurv
from sklearn.model_selection import KFold

D:\Anaconda\envs\lstm3.6pre\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#设置种子
# se = random.randint(0,9999)
se = 1100
np.random.seed(se)
_ = torch.manual_seed(se)
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=5)

In [3]:
#导入数据
dir_path = os.path.dirname('C:\\Users\\gy501\\Desktop\\ReCircle - 英文论文\\ReCircle\\Banded\\')
path = os.path.abspath(os.path.join(dir_path, 'metabric.csv'))
df = pd.read_csv(path, index_col=False)
ci = 0

In [6]:
for train_index, test_index in kf.split(df):
    df_train = []
    df_val = []
    df_test = []
    df_train = df.iloc[train_index]
    df_test = df.iloc[test_index]
    df_val = df_train.sample(frac=0.2)
    df_train = df_train.drop(df_val.index)
    
    #处理数据
    cols_standardize = ['x0', 'x1', 'x2', 'x3', 'x8']
    cols_leave = ['x4', 'x5', 'x6', 'x7']

    standardize = [([col], StandardScaler()) for col in cols_standardize]
    leave = [(col, None) for col in cols_leave]

    x_mapper = DataFrameMapper(standardize + leave)

    x_train = x_mapper.fit_transform(df_train).astype('float32')
    x_val = x_mapper.transform(df_val).astype('float32')
    x_test = x_mapper.transform(df_test).astype('float32')
    
    num_durations = 10
    #labtrans = DeepHitSingle.label_transform(num_durations)
    labtrans = BandedDeepHitSingle.label_transform(num_durations)
    get_target = lambda df: (df['duration'].values, df['event'].values)
    y_train = labtrans.fit_transform(*get_target(df_train))
    y_val = labtrans.transform(*get_target(df_val))
    
    train = (x_train, y_train)
    val = (x_val, y_val)
    # We don't need to transform the test labels
    durations_test, events_test = get_target(df_test)
    
    in_features = x_train.shape[1]
    num_nodes = [32, 32]
    out_features = labtrans.out_features
    batch_norm = True
    dropout = 0.2
    net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm, dropout)
    a = 0.2
    s = 0.2
    model = BandedDeepHitSingle(net, tt.optim.Adam, alpha=a, sigma=s, duration_index=labtrans.cuts)
    batch_size = 256
    lr_finder = model.lr_finder(x_train, y_train, batch_size, tolerance=3)
    model.optimizer.set_lr(lr_finder.get_best_lr())
    # model.optimizer.set_lr(0.01)
    
    epochs = 256
    callbacks = [tt.callbacks.EarlyStopping()]
    verbose = False
    log = model.fit(x_train, y_train, batch_size, epochs, callbacks,verbose,val_data=val)
    
    surv = model.interpolate(10).predict_surv_df(x_test)
    ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
    print(ev.concordance_td('antolini'))
    ci += ev.concordance_td('antolini')
    print(ci)

0.6646946968096344
0.6646946968096344
0.6434970014992504
1.3081916983088848
0.6427298240291263
1.9509215223380112
0.6952554015735453
2.6461769239115567
0.6929917333966459
3.3391686573082024


In [7]:
ci /= k
ci

0.6678337314616405

In [8]:
# #训练结果
# surv = model.predict_surv_df(x_test)
# surv.iloc[:, :5].plot(drawstyle='steps-post')
# plt.ylabel('S(t | x)')
# _ = plt.xlabel('Time')

In [9]:
# surv = model.interpolate(10).predict_surv_df(x_test)
# surv.iloc[:, :5].plot(drawstyle='steps-post')
# plt.ylabel('S(t | x)')
# _ = plt.xlabel('Time')
# plt.savefig("METABRIC"+str(5)+"-"+str(5)+".png")

In [10]:
# ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
# ci = ev.concordance_td('antolini')
# time_grid = np.linspace(durations_test.min(), durations_test.max(), 100)
# ibs = ev.integrated_brier_score(time_grid)
# nbll = ev.integrated_nbll(time_grid) 
# ci